In [ ]:
import requests
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, GRU, Dense, Dropout, Input
from sklearn.preprocessing import MinMaxScaler

# Weather API Key
API_KEY = "bd5e378503939ddaee76f12ad7a97608"

# States to analyze
states = ["Maharashtra", "Delhi", "Tamil Nadu", "Karnataka", "West Bengal"]

# Dummy coordinates for states (latitude, longitude)
state_coords = {
    "Maharashtra": (19.7515, 75.7139),
    "Delhi": (28.6139, 77.2090),
    "Tamil Nadu": (11.1271, 78.6569),
    "Karnataka": (15.3173, 75.7139),
    "West Bengal": (22.9868, 87.8550)
}

# Load historical COVID-19 + climate merged dataset
df = pd.read_csv("historical_climate_covid_india.csv")

# Fill missing values
df = df.ffill().fillna(0)

# Create rolling features
for col in ['infected', 'recovered', 'deaths']:
    df[f'{col}_roll7'] = df.groupby('state')[col].transform(lambda x: x.rolling(7).mean().bfill())

# One-hot encode state column
df = pd.get_dummies(df, columns=['state'])

# Features (excluding wind_speed)
feature_cols = [
    'infected', 'recovered', 'deaths',
    'infected_roll7', 'recovered_roll7', 'deaths_roll7',
    'temp', 'humidity', 'precip'
]

# Add one-hot encoded state columns
feature_cols += list(df.columns[df.columns.str.startswith("state_")])

# Normalize
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(df[feature_cols])
X = []
y = []

# Prepare sequences
seq_length = 10
for i in range(len(scaled_data) - seq_length):
    X.append(scaled_data[i:i + seq_length])
    y.append(scaled_data[i + seq_length][:3])  # Predict: infected, recovered, deaths

X = np.array(X)
y = np.array(y)

# Build model
model = Sequential([
    Input(shape=(seq_length, len(feature_cols))),
    Conv1D(filters=64, kernel_size=3, activation='relu'),
    GRU(64),
    Dropout(0.2),
    Dense(3)  # Output: infected, recovered, deaths
])

model.compile(optimizer='adam', loss='mse')
model.fit(X, y, epochs=20, batch_size=32, verbose=1)

# Function to get current weather for a given state
def get_weather(lat, lon):
    url = f"https://api.openweathermap.org/data/2.5/weather?lat={lat}&lon={lon}&appid={API_KEY}&units=metric"
    response = requests.get(url).json()
    return {
        "temp": response["main"]["temp"],
        "humidity": response["main"]["humidity"],
        "precip": response.get("rain", {}).get("1h", 0.0)  # Use 0 if no rain data
    }

# Make predictions for each state
results = []
for state in states:
    print(f"Processing {state}...")
    lat, lon = state_coords[state]
    weather = get_weather(lat, lon)

    # Prepare last sequence from training data
    latest_seq = df[df[f'state_{state}'] == 1].iloc[-seq_length:].copy()

    for col in ['temp', 'humidity', 'precip']:
        latest_seq[col] = weather[col]

    input_seq = scaler.transform(latest_seq[feature_cols])
    input_seq = np.expand_dims(input_seq, axis=0)

    y_pred_scaled = model.predict(input_seq)[0]
    y_pred_scaled = np.maximum(y_pred_scaled, 0)

    # Reverse scale only for output targets
    dummy_input = np.zeros((1, len(feature_cols)))
    dummy_input[0, :3] = y_pred_scaled
    unscaled_output = scaler.inverse_transform(dummy_input)[0][:3]

    results.append({
        "State": state,
        "Predicted_Infected": int(unscaled_output[0]),
        "Predicted_Recovered": int(unscaled_output[1]),
        "Predicted_Deaths": int(unscaled_output[2])
    })

# Save predictions
result_df = pd.DataFrame(results)
print("\n=== Predictions ===")
print(result_df)
result_df.to_csv("covid_predictions_statewise.csv", index=False)
print("\nSaved results to covid_predictions_statewise.csv")

